<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/VLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.0 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update('jax_enable_x64',True)

from scipy.optimize import root_scalar,root
from plotly.subplots import make_subplots

In [3]:
p=Props(['Ethanol','Water'])
z = jnp.array([1/2,1/2])

In [4]:
def bubblePy_ideal(x, T):
    Pi = x*p.Pvap(T)
    bubbleP = np.sum(Pi)
    return bubbleP, Pi/bubbleP

In [5]:
@jax.jit
def bubbleTy_ideal_eq(x, T, P):
    return jnp.sum(x*p.Pvap(T)/P) - 1.

bubbleTy_ideal_eq_gradT = jax.jit(jax.grad(bubbleTy_ideal_eq, 1))
bubbleTy_ideal_eq_grad2T = jax.jit(jax.grad(bubbleTy_ideal_eq_gradT, 1))



def bubbleTy_ideal(x, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(x*Tb)

    res = root_scalar(lambda T: bubbleTy_ideal_eq(x,T,P), x0=Tguess,
                fprime=lambda T: bubbleTy_ideal_eq_gradT(x,T,P),
                fprime2=lambda T: bubbleTy_ideal_eq_grad2T(x,T,P),
                      method='halley')
    if not(res.converged):
        print(res)
    bubbleT = res.root

    return bubbleT, x*p.Pvap(bubbleT)/P

def dewPx_ideal(y, T):
    y_div_Psat = y/p.Pvap(T)
    dewP = 1./jnp.sum(y_div_Psat)
    return dewP, y_div_Psat*dewP

@jax.jit
def dewTx_ideal_eq(y, T, P):
    return jnp.sum(y*P/p.Pvap(T)) - 1.

dewTx_ideal_eq_gradT = jax.jit(jax.grad(dewTx_ideal_eq, 1))
dewTx_ideal_eq_grad2T = jax.jit(jax.grad(dewTx_ideal_eq_gradT, 1))

def dewTx_ideal(y, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(y*Tb)
    res = root_scalar(lambda T: dewTx_ideal_eq(y,T,P), x0=Tguess,
                fprime=lambda T: dewTx_ideal_eq_gradT(y,T,P),
                fprime2=lambda T: dewTx_ideal_eq_grad2T(y,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    dewT = res.root

    return dewT, y*P/p.Pvap(dewT)

In [6]:
dewTx_ideal(z, 2e5)

(Array(383.6969013, dtype=float64),
 Array([0.31438693, 0.68561307], dtype=float64))

In [7]:
#NRTL Parameters: Tij = Aij + Bij/T + Cij * Ln(T) + Dij * T (T Deg K)

In [8]:
def NRTL(x, T):

    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
    G = jnp.exp(- p.NRTL_alpha*tau)
    tauG = tau*G

    # xG_einsum = np.einsum('k, ki -> i', x, G)
    # xtauG_einsum = np.einsum('k, ki -> i', x, tauG)
    xG = x @ G
    xtauG = x @ tauG
    xtauG_xG = xtauG / xG
    return jnp.exp(xtauG_xG +  x@  ((G * (tau - xtauG_xG))/xG).T)


In [9]:
def nGexRT(n, T):

    ntot = jnp.sum(n)
    x = n/ntot

    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
    G = jnp.exp(- p.NRTL_alpha*tau)
    tauG = tau*G
    xG = x @ G
    xtauG = x @ tauG
    xtauG_xG = xtauG / xG

    return ntot*jnp.sum(x * xtauG_xG)

lngamma=jax.grad(nGexRT, 0)

def NRTL_Gex(z,T):

    return jnp.exp(lngamma(z,373.15))


In [10]:
@jax.jit
def bubbleTy_eq(x, T, P):
    return jnp.sum(x* NRTL(x,T) * p.Pvap(T)/P) - 1.

bubbleTy_eq_gradT = jax.jit(jax.grad(bubbleTy_eq, 1))
bubbleTy_eq_grad2T = jax.jit(jax.grad(bubbleTy_eq_gradT, 1))



def bubbleTy(x, P):
    T_ideal, y_ideal = bubbleTy_ideal(x, P)

    res = root_scalar(lambda T: bubbleTy_eq(x,T,P), x0=T_ideal,
                fprime=lambda T: bubbleTy_eq_gradT(x,T,P),
                fprime2=lambda T: bubbleTy_eq_grad2T(x,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    bubbleT = res.root

    return bubbleT, x*NRTL(x,bubbleT)*p.Pvap(bubbleT)/P



In [11]:

# @jax.jit
def dewTx_eq(y, vec, P):
    T = vec[0]
    x = vec[1:]**2
    return jnp.r_[x*NRTL(x,T)*p.Pvap(T)/P - y, jnp.sum(x)-1]

dewTx_eq_jac = jax.jacobian(dewTx_eq, 1 )

In [12]:
def dewTx(y, P, guess=None):

    if guess is None:
        dewT_guess, dewx_guess = dewTx_ideal(y,P)
    else:
        dewT_guess, dewx_guess = guess

    res=root(lambda vec: dewTx_eq(y, vec, P),
        jnp.r_[dewT_guess, dewx_guess],
        jac=lambda vec: dewTx_eq_jac(y, vec, P))

    if not(res.success):
        print(res)
    # print(res)

    dewT = res.x[0]
    dewx = res.x[1:]**2

    return dewT, dewx

In [13]:
y1 = np.linspace(0,1.,25)
ys = np.c_[y1, 1-y1]
dewTs = []
dewxs = []

In [14]:
guess = None
for y in ys:
    dewT, dewx = dewTx(y, 101325, guess)
    guess = (dewT, dewx)
    print(y, dewT, dewx)
    dewTs.append(dewT)
    dewxs.append(dewx)

[0. 1.] 373.1478177065466 [0. 1.]
[0.04166667 0.95833333] 372.06438916945154 [0.00384654 0.99615346]
[0.08333333 0.91666667] 370.94616492415344 [0.00815908 0.99184092]
[0.125 0.875] 369.7909221359241 [0.01303852 0.98696148]
[0.16666667 0.83333333] 368.596320567221 [0.01861859 0.98138141]
[0.20833333 0.79166667] 367.3599509907055 [0.02508081 0.97491919]
[0.25 0.75] 366.0794341521504 [0.0326786 0.9673214]
[0.29166667 0.70833333] 364.75261317718395 [0.04177797 0.95822203]
[0.33333333 0.66666667] 363.37792564032776 [0.05292971 0.94707029]
[0.375 0.625] 361.9551350183087 [0.06700617 0.93299383]
[0.41666667 0.58333333] 360.48682274344833 [0.08547979 0.91452021]
[0.45833333 0.54166667] 358.9816042304736 [0.11103904 0.88896096]
[0.5 0.5] 357.46152607673855 [0.14903094 0.85096906]
[0.54166667 0.45833333] 355.97937152693805 [0.21029961 0.78970039]
[0.58333333 0.41666667] 354.6454608328755 [0.30787477 0.69212523]
[0.625 0.375] 353.5853916923284 [0.42503266 0.57496734]
[0.66666667 0.33333333] 352.

In [15]:
dewxs=np.vstack(dewxs)

In [16]:
fig = make_subplots()
fig.add_scatter(x=dewxs[:,0], y=y1, mode='lines')
fig.add_scatter(x=[0,1],y=[0,1],mode='lines')
fig.update_layout(width=600, height=600, template='plotly_dark')

In [17]:
fig2 = make_subplots()
fig2.add_scatter(x=dewxs[:,0], y=dewTs, mode='lines')
fig2.add_scatter(x=y1, y=dewTs, mode='lines')
fig2.update_layout(width=600, height=600, template='plotly_dark')

In [28]:
#Flash Calculation
F = np.array([60., 40.])
z= F/np.sum(F)
flashP = 101325
bubbleT = bubbleTy(z,flashP)[0]
dewT = dewTx(z,flashP)[0]

flashT = (bubbleT + dewT)/2  #set our flash drum to middle of 2 phase region temperature wise

In [48]:
def flashTP_eq(vec, T, P, F):
    Ftotal = jnp.sum(F)
    L,V = jnp.split(vec**2,2)
    x = L/jnp.sum(L)
    y= V/jnp.sum(V)

    return jnp.r_[x*NRTL(x,T)*p.Pvap(T)/P - y, (L+V - F)/Ftotal]

flashTP_eq_jac=jax.jacobian(flashTP_eq,0)

def flashTP(F, T, P):
    guess= np.r_[F/2, F/2]
    res=root(lambda vec: flashTP_eq(vec, flashT, flashP, F), guess, jac=lambda vec: flashTP_eq_jac(vec, flashT, flashP, F))
    if not(res.success):
        res
    return jnp.split(res.x**2,2)

In [49]:
flashTP(F, flashT, flashP)

[Array([10.59305956, 12.1971728 ], dtype=float64),
 Array([49.40694043, 27.8028272 ], dtype=float64)]

In [71]:
def flashPQ_eq(vec, P, Q, F, enthalpyF):
    Ftotal = jnp.sum(F)
    T = vec[0]**2
    L,V = jnp.split(vec[1:]**2,2)
    x = L/jnp.sum(L)
    y= V/jnp.sum(V)

    return jnp.r_[x*NRTL(x,T)*p.Pvap(T)/P - y,   (L+V - F)/Ftotal, (p.Hl(L, T) + p.Hv(V,T) - enthalpyF - Q)/enthalpyF]

flashPQ_eq_jac = jax.jacobian(flashPQ_eq, 0)

def flashPQ(F, feedT, feedP, flashP, Q=0.):
    feedL, feedV = flashTP(F, feedT, feedP)
    enthalpyF = p.Hl(feedL, feedT) + p.Hv(feedV, feedT)
    guess = np.sqrt(np.r_[feedT, F/2, F/2])
    res=root(lambda vec: flashPQ_eq(vec, flashP, Q, F, enthalpyF), guess,
        jac=lambda vec: flashPQ_eq_jac(vec, flashP, 0., F, enthalpyF))

    if not(res.success):
        print(res)

    return res.x[0]**2, np.split(res.x[1:]**2,2)


In [74]:
#Flash Calculation
p=Props(['Acetone', 'Ethanol','Isopropanol', 'Water'])
F = np.array([25, 25, 30., 40.])
z= F/np.sum(F)
feedP = 3e5
bubbleT = bubbleTy(z,feedP)[0]
dewT = dewTx(z,feedP)[0]
feedT= (bubbleT+dewT)/2
feedL, feedV = flashTP(F, feedT, feedP)
enthalpyF = p.Hl(feedL, feedT) + p.Hv(feedV, feedT)
flashP = 1e5

In [75]:
flashPQ(F, feedT, feedP, flashP, 0.)

 message: The iteration is not making good progress, as measured by the 
            improvement from the last five Jacobian evaluations.
 success: False
  status: 4
     fun: [ 2.979e-04  1.200e-03  6.455e-04  1.494e-03 -5.877e-03
           -4.558e-03 -4.607e-03 -3.711e-03  4.422e-03]
       x: [ 1.869e+01  9.453e-03  1.863e-02  1.979e-02  3.335e-02
            4.929e+00  4.945e+00  5.426e+00  6.289e+00]
    nfev: 46
    njev: 12
    fjac: [[-2.970e-01 -4.157e-01 ...  0.000e+00  1.331e-02]
           [ 9.523e-01 -1.648e-01 ...  0.000e+00  2.429e-03]
           ...
           [ 1.623e-04  6.217e-04 ... -4.934e-01  1.130e-01]
           [-6.078e-04 -2.337e-03 ...  5.455e-01 -4.122e-01]]
       r: [-7.444e-01 -5.932e+00 ... -3.964e-02  1.314e-02]
     qtf: [-1.929e-03 -1.156e-03 -1.029e-03 -4.216e-04 -9.615e-04
           -2.458e-03 -7.666e-04 -4.527e-04 -1.012e-02]


(349.3374368959457,
 [array([8.93574216e-05, 3.47142890e-04, 3.91496917e-04, 1.11204530e-03]),
  array([24.29465408, 24.45269207, 29.44680923, 39.55354043])])